#Fast OMP Implementation

In [ ]:
%load_ext Cython

In [ ]:
%%cython

"""Cython allows us to call lower level c-code instead of using Python. It can be a surprisingly big speedup!"""
import cython
from scipy.linalg.cython_blas cimport idamax, isamax, daxpy, dgemv, dtrmv, dcopy
from scipy.linalg.cython_lapack cimport dposv, dppsv, sppsv

ctypedef fused proj_t:
    double
    float

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void ppsv(proj_t[:, :] As,
           proj_t[:, :, :] ys) nogil:
    # Works not for strided array I think. And please do not give a negative-stride array.
    cdef Py_ssize_t B = ys.shape[0]  # Batch size
    cdef int N = ys.shape[1]
    cdef int nrhs = ys.shape[2]
    cdef int info = 0  # Just discard any error signals ;)
    cdef char uplo = 85 # The letter 'U', since we store the lower triangle and fortran sees As.T.
    # cdef int ldb = ys[0].strides[0] // sizeof(double)

    for i from 0 <= i < B:
        if proj_t is double:  # One C-function is created for each of these specializations! :) (see argmax_blast.__signatures__)
            dppsv(&uplo, &N, &nrhs, &As[i, 0], &ys[i, 0, 0], &N, &info)
        elif proj_t is float:
            sppsv(&uplo, &N, &nrhs, &As[i, 0], &ys[i, 0, 0], &N, &info)


@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void argmax_blast(proj_t[:, :] projections,
                 long long[:] output) nogil:
    # TODO: Numpy has its own indexing data-type - this may be a more appropriate output, and may even be faster.
    # http://conference.scipy.org/static/wiki/seljebotn_cython.pdf
    # https://apprize.best/python/cython/3.html
    cdef Py_ssize_t B = projections.shape[0]
    cdef int N = projections.shape[1]
    cdef int incx = projections.strides[1] // sizeof(proj_t)  # Stride between elements.
    cdef Py_ssize_t i
    for i from 0 <= i < B:
        if proj_t is double:
            output[i] = idamax(&N, &projections[i, 0], &incx) - 1
        elif proj_t is float:
            output[i] = isamax(&N, &projections[i, 0], &incx) - 1

In [ ]:
"""This cell contains the code we've implemented. You should be able to call each function directly, or alternatively, see our example calls below"""

import os
import torch
import torch.utils
import torch.utils.data
from sklearn.datasets import make_sparse_coded_signal
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from contextlib import contextmanager
from timeit import default_timer
# from test_omp import omp_naive
# from test import *  # FIXME: better name
# from line_profiler import line_profiler

n_components, n_features = 100, 100
n_nonzero_coefs = 17
n_samples = 50

@contextmanager
def elapsed_timer():
    # https://stackoverflow.com/questions/7370801/how-to-measure-elapsed-time-in-python
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start


def run_omp(X, y, n_nonzero_coefs, precompute=True, tol=0.0, normalize=False, fit_intercept=False, alg='naive'):
    if not isinstance(X, torch.Tensor):
        X = torch.as_tensor(X)
        y = torch.as_tensor(y)

    # We can either return sets, (sets, solutions), or xests
    # These are all equivalent, but are simply more and more dense representations.
    # Given sets and X and y one can (re-)construct xests. The second is just a sparse vector repr.

    # https://github.com/scikit-learn/scikit-learn/blob/15a949460dbf19e5e196b8ef48f9712b72a3b3c3/sklearn/linear_model/_omp.py#L690
    if fit_intercept or normalize:
        X = X.clone()
        assert not isinstance(precompute, torch.Tensor), "If user pre-computes XTX they can also pre-normalize X" \
                                                         " as well, so normalize and fit_intercept must be set false."

    if fit_intercept:
        X = X - X.mean(0)
        y = y - y.mean(1)[:, None]

    # To keep a good condition number on X, especially with Cholesky compared to LU factorization,
    # we should probably always normalize it (OMP is invariant anyways)
    if normalize is True:  # User can also just optionally supply pre-computed norms.
        normalize = (X * X).sum(0).sqrt()
        X /= normalize[None, :]

    if precompute is True or alg == 'v0':
        precompute = X.T @ X

    # If n_nonzero_coefs is equal to M, one should just return lstsq
    if alg == 'naive':
        sets, solutions, lengths = omp_naive(X, y, n_nonzero_coefs=n_nonzero_coefs, XTX=precompute, tol=tol)
    elif alg == 'v0':
        sets, solutions, lengths = omp_v0(X, y, n_nonzero_coefs=n_nonzero_coefs, XTX=precompute, tol=tol)


    solutions = solutions.squeeze(-1)
    if normalize is not False:
        solutions /= normalize[sets]

    xests = y.new_zeros(y.shape[0], X.shape[1])
    if lengths is None:
        xests[torch.arange(y.shape[0], dtype=sets.dtype, device=sets.device)[:, None], sets] = solutions
    else:
        for i in range(y.shape[0]):
            # xests[i].scatter_(-1, sets[i, :lengths[i]], solutions[i, :lengths[i]])
            xests[i, sets[i, :lengths[i]]] = solutions[i, :lengths[i]]

    return xests

def batch_mm(matrix, matrix_batch, return_contiguous=True):
    """
    :param matrix: Sparse or dense matrix, size (m, n).
    :param matrix_batch: Batched dense matrices, size (b, n, k).
    :return: The batched matrix-matrix product, size (m, n) x (b, n, k) = (b, m, k).
    """
    # One dgemm is faster than many dgemv.
    # From https://github.com/pytorch/pytorch/issues/14489#issuecomment-607730242
    batch_size = matrix_batch.shape[0]
    # Stack the vector batch into columns. (b, n, k) -> (n, b, k) -> (n, b*k)
    vectors = matrix_batch.transpose([1, 0, 2]).reshape(matrix.shape[1], -1)

    # A matrix-matrix product is a batched matrix-vector product of the columns.
    # And then reverse the reshaping. (m, n) x (n, b*k) = (m, b*k) -> (m, b, k) -> (b, m, k)
    if return_contiguous:
        result = np.empty_like(matrix_batch, shape=(batch_size, matrix.shape[0], matrix_batch.shape[2]))
        np.matmul(matrix, vectors, out=result.transpose([1, 0, 2]).reshape(matrix.shape[0], -1))
    else:
        result = (matrix @ vectors).reshape(matrix.shape[0], batch_size, -1).transpose([1, 0, 2])

    return result


def innerp(x, y=None, out=None):
    if y is None:
        y = x
    if out is not None:
        out = out[:, None, None]  # Add space for two singleton dimensions.
    return torch.matmul(x[..., None, :], y[..., :, None], out=out)[..., 0, 0]

def cholesky_solve(ATA, ATy):
    if ATA.dtype == torch.half or ATy.dtype == torch.half:
        return ATy.to(torch.float).cholesky_solve(torch.cholesky(ATA.to(torch.float))).to(ATy.dtype)
    return ATy.cholesky_solve(torch.cholesky(ATA)).to(ATy.dtype)


def omp_naive(X, y, n_nonzero_coefs, tol=None, XTX=None):
    on_cpu = not (y.is_cuda or y.dtype == torch.half)
    # torch.cuda.synchronize()
    # Given X as an MxN array and y as an BxN array, do omp to approximately solve Xb=y

    # Base variables
    XT = X.contiguous().t()  # Store XT in fortran-order.
    y = y.contiguous()
    r = y.clone()

    sets = y.new_zeros((n_nonzero_coefs, y.shape[0]), dtype=torch.long).t()
    if tol:
        result_sets = sets.new_zeros(y.shape[0], n_nonzero_coefs)
        result_lengths = sets.new_zeros(y.shape[0])
        result_solutions = y.new_zeros((y.shape[0], n_nonzero_coefs, 1))
        original_indices = torch.arange(y.shape[0], dtype=sets.dtype, device=sets.device)

    # Trade b*k^2+bk+bkM = O(bkM) memory for much less compute time. (This has to be done anyways since we are batching,
    # otherwise one could just permute columns of X in-place as in https://github.com/scikit-learn/scikit-learn/blob/15a949460dbf19e5e196b8ef48f9712b72a3b3c3/sklearn/linear_model/_omp.py#L28 )
    ATs = y.new_zeros(r.shape[0], n_nonzero_coefs, X.shape[0])
    ATys = y.new_zeros(r.shape[0], n_nonzero_coefs, 1)
    ATAs = torch.eye(n_nonzero_coefs, dtype=y.dtype, device=y.device)[None].repeat(r.shape[0], 1, 1)
    if on_cpu:
        # For CPU it is faster to use a packed representation of the lower triangle in ATA.
        tri_idx = torch.tril_indices(n_nonzero_coefs, n_nonzero_coefs, device=sets.device, dtype=sets.dtype)
        ATAs = ATAs[:, tri_idx[0], tri_idx[1]]

    solutions = y.new_zeros((r.shape[0], 0))

    for k in range(n_nonzero_coefs+bool(tol)):
        # STOPPING CRITERIA
        if tol:
            problems_done = innerp(r) <= tol
            if k == n_nonzero_coefs:
                problems_done[:] = True

            if problems_done.any():
                remaining = ~problems_done

                orig_idxs = original_indices[problems_done]
                result_sets[orig_idxs, :k] = sets[problems_done, :k]
                result_solutions[orig_idxs, :k] = solutions[problems_done]
                result_lengths[orig_idxs] = k
                original_indices = original_indices[remaining]

                # original_indices = original_indices[remaining]
                ATs = ATs[remaining]
                ATys = ATys[remaining]
                ATAs = ATAs[remaining]
                sets = sets[remaining]
                y = y[remaining]
                r = r[remaining]
                if problems_done.all():
                    return result_sets, result_solutions, result_lengths
        # GET PROJECTIONS AND INDICES TO ADD
        if on_cpu:
            projections = batch_mm(XT.numpy(), r[:, :, None].numpy())
            argmax_blast(projections.squeeze(-1), sets[:, k].numpy())
        else:
            projections = XT @ r[:, :, None]
            sets[:, k] = projections.abs().sum(-1).argmax(-1)  # Sum is just a squeeze, but would be relevant in SOMP.

        # UPDATE AT
        AT = ATs[:, :k + 1, :]
        updateA = XT[sets[:, k], :]
        AT[:, k, :] = updateA

        # UPDATE ATy based on AT
        ATy = ATys[:, :k + 1]
        innerp(updateA, y, out=ATy[:, k, 0])

        # UPDATE ATA based on AT or precomputed XTX.
        if on_cpu:
            packed_idx = k * (k - 1) // 2
            if XTX is not None:  # Update based on precomputed XTX.
                ATAs.t()[k + packed_idx:packed_idx + 2 * k + 1, :].t().numpy()[:] = XTX[sets[:, k, None], sets[:, :k + 1]]
            else:
                np.matmul(AT[:, :k + 1, :].numpy(), updateA[:, :, None].numpy(),
                          out=ATAs.t()[k + packed_idx:packed_idx + 2 * k + 1, :].t()[:, :, None].numpy())
        else:
            ATA = ATAs[:, :k + 1, :k + 1]
            if XTX is not None:
                ATA[:, k, :k + 1] = XTX[sets[:, k, None], sets[:, :k + 1]]
            else:
                # Update ATAs by adding the new column of inner products.
                torch.bmm(AT[:, :k + 1, :], updateA[:, :, None], out=ATA[:, k, :k + 1, None])

        # SOLVE ATAx = ATy.
        if on_cpu:
            solutions = ATy.permute(0, 2, 1).clone().permute(0, 2, 1)  # Get a copy.
            ppsv(ATAs.t()[:packed_idx + 2 * k + 1, :].t().contiguous().numpy(), solutions.numpy())
        else:
            ATA[:, :k, k] = ATA[:, k, :k]  # Copy lower triangle to upper triangle.
            solutions = cholesky_solve(ATA, ATy)

        # FINALLY, GET NEW RESIDUAL r=y-Ax
        if on_cpu:
            np.subtract(y.numpy(), (AT.permute(0, 2, 1).numpy() @ solutions.numpy()).squeeze(-1), out=r.numpy())
        else:
            torch.baddbmm(y[:, :, None], AT.permute(0, 2, 1), solutions, beta=-1, out=r[:, :, None])

    return sets, solutions, None

def omp_v0(X, y, XTX, n_nonzero_coefs=None, tol=None, inverse_cholesky=True):
    B = y.shape[0]
    normr2 = innerp(y)  # Norm squared of residual.
    projections = (X.transpose(1, 0) @ y[:, :, None]).squeeze(-1)
    sets = y.new_zeros(n_nonzero_coefs, B, dtype=torch.int64)

    if inverse_cholesky:
        # Doing the inverse-cholesky iteratively uses more memory,
        # but takes less time than waiting till solving the problem in the end it seems.
        # (Since F is triangular it could be __even faster__ to multiply, prob. not on GPU tho.)
        F = torch.eye(n_nonzero_coefs, dtype=y.dtype, device=y.device).repeat(B, 1, 1)
        a_F = y.new_zeros(n_nonzero_coefs, B, 1)

    D_mybest = y.new_empty(B, n_nonzero_coefs, XTX.shape[0])
    temp_F_k_k = y.new_ones((B, 1))

    if tol:
        result_lengths = sets.new_zeros(y.shape[0])
        result_solutions = y.new_zeros((y.shape[0], n_nonzero_coefs, 1))
        finished_problems = sets.new_zeros(y.shape[0], dtype=torch.bool)

    for k in range(n_nonzero_coefs+bool(tol)):
        # STOPPING CRITERIA
        if tol:
            problems_done = normr2 <= tol
            if k == n_nonzero_coefs:
                problems_done[:] = True

            if problems_done.any():
                new_problems_done = problems_done & ~finished_problems
                finished_problems.logical_or_(problems_done)
                result_lengths[new_problems_done] = k
                if inverse_cholesky:
                    result_solutions[new_problems_done, :k] = F[new_problems_done, :k, :k].permute(0, 2, 1) @ a_F[:k, new_problems_done].permute(1, 0, 2)
                else:
                    assert False, "inverse_cholesky=False with tol != None is not handled yet"
                if problems_done.all():
                    return sets.t(), result_solutions, result_lengths

        sets[k] = projections.abs().argmax(1)
        # D_mybest[:, k, :] = XTX[gamma[k], :]  # Same line as below, but significantly slower. (prob. due to the intermediate array creation)
        torch.gather(XTX, 0, sets[k, :, None].expand(-1, XTX.shape[1]), out=D_mybest[:, k, :])
        if k:
            D_mybest_maxindices = D_mybest.permute(0, 2, 1)[torch.arange(D_mybest.shape[0], dtype=sets.dtype, device=sets.device), sets[k], :k]
            torch.rsqrt(1 - innerp(D_mybest_maxindices),
                        out=temp_F_k_k[:, 0])  # torch.exp(-1/2 * torch.log1p(-inp), temp_F_k_k[:, 0])
            D_mybest_maxindices *= -temp_F_k_k  # minimal operations, exploit linearity
            D_mybest[:, k, :] *= temp_F_k_k
            D_mybest[:, k, :, None].baddbmm_(D_mybest[:, :k, :].permute(0, 2, 1), D_mybest_maxindices[:, :, None])


        temp_a_F = temp_F_k_k * torch.gather(projections, 1, sets[k, :, None])
        normr2 -= (temp_a_F * temp_a_F).squeeze(-1)
        projections -= temp_a_F * D_mybest[:, k, :]
        if inverse_cholesky:
            a_F[k] = temp_a_F
            if k:  # Could maybe get a speedup from triangular mat mul kernel.
                torch.bmm(D_mybest_maxindices[:, None, :], F[:, :k, :], out=F[:, k, None, :])
                F[:, k, k] = temp_F_k_k[..., 0]
    else: # FIXME: else branch will not execute if n_nonzero_coefs=0, so solutions is undefined.
        # Normal exit, used when tolerance=None.
        if inverse_cholesky:
            solutions = F.permute(0, 2, 1) @ a_F.squeeze(-1).transpose(1, 0)[:, :, None]
        else:
            # Solving the problem in the end without using inverse Cholesky.
            AT = X.T[sets.T]
            solutions = cholesky_solve(AT @ AT.permute(0, 2, 1), AT @ y.T[:, :, None])

    return sets.t(), solutions, None

# RoBerta

In [1]:
!pip install transformers
import torch
from torch import nn
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import classification_report
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output
from transformers import logging
logging.set_verbosity_error()

import sklearn
from sklearn.linear_model import OrthogonalMatchingPursuit

from google.colab import drive
drive.mount('/content/drive')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

STATE_PATH = '/content/drive/MyDrive/NLP-Final/roberta_state_w_finetuning'
KEYS_VALUES_PATH = "/content/drive/My Drive/NLP-Final/roberta_sparse_keys&values/"

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 34.8 MB/s 
     |████████████████████████████████| 596 kB 31.9 MB/s 
     |████████████████████████████████| 6.6 MB 24.6 MB/s 
     |████████████████████████████████| 77 kB 3.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Mounted at /content/drive
cuda


In [ ]:
path = '/content/drive/MyDrive/NLP-Final/'

train_data = pd.read_csv(path + 'train.csv')
test_data = pd.read_csv(path + 'test.csv')

train_data = train_data[:2000]
test_data = test_data[:500]

train_data = train_data.to_dict(orient='records')
test_data = test_data.to_dict(orient='records')
type(train_data)

train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), test_data)))

len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(2000, 2000, 500, 500)

In [ ]:
BATCH_SIZE = 16

def process_data(tokenizer):
    train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train_texts))
    test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], test_texts))
    train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
    test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")

    train_y = np.array(train_labels) == 'pos'
    test_y = np.array(test_labels) == 'pos'
    # test_y = np.append(test_y, np.array(test_labels) != 'pos')

    train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
    test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
    
    train_tokens_tensor = torch.tensor(train_tokens_ids)
    train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

    test_tokens_tensor = torch.tensor(test_tokens_ids)
    test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

    train_masks_tensor = torch.tensor(train_masks)
    test_masks_tensor = torch.tensor(test_masks)

    train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

    test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)
    
    return train_dataloader, test_dataloader, test_y

In [ ]:
EPOCHS = 10

def train_model(model, train_dataloader):
  param_optimizer = list(model.named_parameters()) 
  optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
  optimizer = Adam(model.parameters(), lr=3e-6)
  torch.cuda.empty_cache()   # Clearing Cache space for a fresh Model run

  for epoch_num in range(EPOCHS):
    model.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')

        tmp = torch.eq(labels, torch.zeros_like(labels))
        labels_for_loss = torch.cat((labels, tmp), dim=1)
        
        logits = model(token_ids, masks).logits
        
        loss_func = nn.CrossEntropyLoss()
        batch_loss = loss_func(logits, labels_for_loss)
        train_loss += batch_loss.item()
        
        model.zero_grad()
        batch_loss.backward()

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))
  return model

In [ ]:
def eval_model(model, test_dataloader, test_y):
  model_predicted = []
  all_logits = []
  with torch.no_grad():
      for step_num, batch_data in enumerate(test_dataloader):
          token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
          tmp = torch.eq(labels, torch.zeros_like(labels))
          labels_for_loss = torch.cat((labels, tmp), dim=1)

          logits = model(token_ids, masks).logits

          loss_func = nn.CrossEntropyLoss()
          loss = loss_func(logits, labels_for_loss)
          numpy_logits = logits.cpu().detach().numpy()
          
          model_predicted += list(numpy_logits[:, 0] > 0.5)
          all_logits += list(numpy_logits[:, 0])

          token_ids, masks, labels = tuple(t.to("cpu") for t in batch_data) # PATCH- move back to cpu
  print(type(test_y)) 
  print(type(model_predicted))        
  return classification_report(test_y, model_predicted, output_dict=True)

In [ ]:
# roberta without finetuning
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_dataloader, test_dataloader, test_y = process_data(roberta_tokenizer)
roberta = RobertaForSequenceClassification.from_pretrained('roberta-base')
report = eval_model(roberta, test_dataloader, test_y)
print(report)

In [ ]:
# finetune Roberta
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_dataloader, test_dataloader, test_y = process_data(roberta_tokenizer)
roberta = RobertaForSequenceClassification.from_pretrained('roberta-base')
roberta = roberta.cuda()
roberta_w_finetuning = train_model(roberta, train_dataloader)
report = eval_model(roberta_w_finetuning, test_dataloader, test_y)
print(report)

STATE_PATH = '/content/drive/MyDrive/NLP-Final/roberta_state_w_finetuning'
roberta_st = roberta_w_finetuning.state_dict()
torch.save(roberta_st, STATE_PATH)

In [ ]:
# this is how to load the model
STATE_PATH = '/content/drive/MyDrive/NLP-Final/roberta_state_w_finetuning'
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_dataloader, test_dataloader, test_y = process_data(roberta_tokenizer)
roberta_check = RobertaForSequenceClassification.from_pretrained('roberta-base')
roberta_check = roberta_check.cuda()
roberta_check.load_state_dict(torch.load(STATE_PATH))
# eval_model(roberta_check, test_dataloader, test_y)

In [ ]:
def print_layers_params(model):
  for name,x in model.named_parameters():
    if "intermediate.dense.weight" in name: # intermidiate is the feed forward part of the layer 
      print(name)

#**EXPERIMENTS**

# Sparse Keys & Values Analysis

In [ ]:
from numpy import savetxt
LAYER_NUM = 11

# Load fine-tuned GPT2
roberta_analysis = RobertaForSequenceClassification.from_pretrained('roberta-base')
roberta_analysis = roberta_analysis.cuda()
roberta_analysis.load_state_dict(torch.load(STATE_PATH))

# for name,x in gpt2_analysis.named_parameters():
   # if "mlp.c_fc.weight" in name: # c_fc is the feed forward part of the layer 
      # print(name)

# Get embeddings and BERT state
embeddings = roberta_analysis.roberta.get_input_embeddings()
E = embeddings.weight.detach().to(device)
roberta_analysis_st = roberta_analysis.roberta.state_dict() # Bert model state dictionary

# Compute sparse keys
# keys = gpt2_analysis_st[f"encoder.layer.{LAYER_NUM}.intermediate.dense.weight"].detach().to(device)
# n_non_zero_keys = max(int(0.1 * keys.shape[0]), 1)
# sparse_keys = run_omp(E.T,  keys.T, n_non_zero_keys) # run optimized OMP
# savetxt(KEYS_VALUES_PATH+f"layer{LAYER_NUM}_keys.csv", sparse_keys, delimiter=',')

# Compute sparse values
values = roberta_analysis_st[f"encoder.layer.{LAYER_NUM}.output.dense.weight"].detach().to(device)
n_non_zero_values = max(int(0.1 * values.shape[1]), 1)
sparse_values = run_omp(E.T,  values.T, n_non_zero_values) # run optimized OMP
savetxt(KEYS_VALUES_PATH+f"layer{LAYER_NUM}_values.csv", sparse_values, delimiter=',')

RuntimeError: ignored

In [ ]:
from numpy import loadtxt
LAYER_NUM = 11

sparse_keys = loadtxt(KEYS_VALUES_PATH+f"layer{LAYER_NUM}_keys.csv", delimiter=',').astype("double")
sparse_keys = torch.from_numpy(sparse_keys).to(device)
sparse_values = loadtxt(KEYS_VALUES_PATH+f"layer{LAYER_NUM}_values.csv", delimiter=',').astype("double")
sparse_values = torch.from_numpy(sparse_values).to(device)

In [3]:
from numpy import loadtxt
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
sparse_keys = loadtxt("/content/drive/My Drive/NLP-Final/roberta_sparse_keys/results_omp_roberta_layer11.csv", delimiter=',').astype("double")
sparse_keys = torch.from_numpy(sparse_keys).to(device)

for i in range(100):
  # non_zero_keys = torch.nonzero(sparse_keys[i])
  sorted_keys, indices_keys = torch.sort(torch.abs(sparse_keys[i]), descending=True)
  print(roberta_tokenizer.batch_decode(indices_keys[0:10]))

  # non_zero_values = torch.nonzero(sparse_values[i])
  # sorted_values, indices_values = torch.sort(torch.abs(sparse_values[i]), descending=True)
  # print(roberta_tokenizer.batch_decode(indices_values[0:10]))
  print("#################################################")

['2', ' 12', ' session', ' Pl', ' real', 'ary', ' owned', ' students', ' selling', ' our']
#################################################
[' el', ' seasons', ' change', ' brain', ' simply', ' significant', '<s>', ' Jose', ' citizens', '17']
#################################################
[' struck', ' victim', 'm', ' made', ' kind', ' IN', ' really', ' survey', ' brain', 'ot']
#################################################
[' David', ' brother', ' though', ' De', ' make', ' treated', ' At', ' whole', ' voice', ' struck']
#################################################
[' figures', 'il', ' Star', ' men', 'So', '2018', ' UK', ' Black', ' movement', ' net']
#################################################
[' Cleveland', ' half', ' Florida', 'ty', ' baby', ' above', ' faces', 'ij', ' States', ' find']
#################################################
[' IN', ' inside', ' skin', ' waiting', ' looks', ' focus', ' international', 'q', 'x', ' den']
##################################

# Yuvi - Fast OMP

In [ ]:
STATE_PATH = '/content/drive/MyDrive/NLP-Final/roberta_state_w_finetuning'
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_dataloader, test_dataloader, test_y = process_data(roberta_tokenizer)
roberta_fast_omp = RobertaForSequenceClassification.from_pretrained('roberta-base')
roberta_fast_omp = roberta_fast_omp.cuda()
roberta_fast_omp.load_state_dict(torch.load(STATE_PATH))
roberta_fast_omp.eval()

In [ ]:
emb1 = roberta_fast_omp.roberta.get_input_embeddings() # embeddings
X_tensor = emb1.weight.detach().to(device) 
num_features = X_tensor.shape[1]
n_non_zero = max(int(0.1 * num_features), 1) # 10% of X num of features

roberta_fast_omp_st = roberta_fast_omp.roberta.state_dict() # model state dictionary

torch.cuda.empty_cache() # Clear memory

y_tensor = roberta_fast_omp_st[f"encoder.layer.0.intermediate.dense.weight"].detach().to(device) # get weights for 0 ff layer

xes = run_omp(X_tensor.T,  y_tensor, n_non_zero) # run fast OMP
temp_weights = xes @ X_tensor # matrix mult to get the weights

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:112: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).transpose(-2, -1).conj().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1285.)


In [ ]:
from numpy import savetxt, loadtxt
SPARSE_KEYS_DIR = '/content/drive/MyDrive/NLP-Final/roberta_sparse_keys/'
STATE_PATH = '/content/drive/MyDrive/NLP-Final/roberta_state_w_finetuning'

xes2 = loadtxt(SPARSE_KEYS_DIR+"results_omp_roberta_layer0.csv", delimiter=',').astype("double")
xes2 = torch.from_numpy(xes2).to(device)

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_dataloader, test_dataloader, test_y = process_data(roberta_tokenizer)

# global roberta
roberta_global = RobertaForSequenceClassification.from_pretrained('roberta-base')
roberta_global.load_state_dict(torch.load(STATE_PATH))

emb1 = roberta_global.roberta.get_input_embeddings() # embeddings
X_tensor = emb1.weight.detach().to(device) 
num_features = X_tensor.shape[1]
n_non_zero = max(int(0.1 * num_features), 1) # 10% of X num of features

temp_weights = xes2.T @ X_tensor.T # matrix mult to get the weights

roberta_global_st = roberta_global.roberta.state_dict() # model state dictionary
y_tensor = roberta_global_st[f"encoder.layer.0.intermediate.dense.weight"].detach().to(device) # get weights for ith ff layer

roberta_global_st[f"encoder.layer.{i}.intermediate.dense.weight"]  = temp_weights
roberta_global.roberta.load_state_dict(roberta_global_st)

report = eval_model(roberta_global, test_dataloader, test_y)
print(report)

RuntimeError: ignored

# Yuvi SKlearn Test

In [ ]:
# Load Roberta after finetuning
STATE_PATH = '/content/drive/MyDrive/NLP-Final/roberta_state_w_finetuning'
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_dataloader, test_dataloader, test_y = process_data(roberta_tokenizer)
roberta1 = RobertaForSequenceClassification.from_pretrained('roberta-base')
roberta1 = roberta1.cuda()
roberta1.load_state_dict(torch.load(STATE_PATH))
# eval_model(roberta1, test_dataloader, test_y)
# roberta1.load_state_dict(torch.load(STATE_PATH, map_location=torch.device('cpu')))

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

<All keys matched successfully>

In [ ]:
from sklearn.linear_model import OrthogonalMatchingPursuit

emb1 = roberta1.roberta.get_input_embeddings() # embeddings
X = emb1.weight.detach()
num_features = X.shape[1]
n_non_zero = max(int(0.1 * num_features), 1) # 10% of X num of features

roberta1_st = roberta1.roberta.state_dict() # model state dictionary

torch.cuda.empty_cache() # Clear memory

y = roberta1_st[f"encoder.layer.0.intermediate.dense.weight"].detach() # get weights for 0 ff layer
reg = OrthogonalMatchingPursuit(normalize=False).fit(X.T.cpu(),  y.T.cpu())
xes = reg.predict(X.T)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_omp.py:420: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  X, y[:, k], n_nonzero_coefs, tol, copy_X=copy_X, return_path=return_path


In [ ]:
from numpy import savetxt, loadtxt
savetxt(f"/content/drive/MyDrive/NLP-Final/results_omp_roberta_layer0.csv", xes, delimiter=',')

# Replace every state untouching the others

In [ ]:
from sklearn.linear_model import OrthogonalMatchingPursuit

# Replace every state untouching the others
emb1 = roberta1.roberta.get_input_embeddings() # embeddings
roberta1_st = roberta1.roberta.state_dict() # Bert model state dictionary

X_tensor = emb1.weight.detach().to(device) 
num_features = X_tensor.shape[1]
n_non_zero = max(int(0.1 * num_features), 1) # 10% of X num of features

exp_results = dict()

for i in range(13):
  torch.cuda.empty_cache() # Clear memory
  roberta1.load_state_dict(torch.load(STATE_PATH, map_location=torch.device('cpu'))) # load first the original weights

  if i == 0:
    # print("Results Before Replacement")
    # res = eval_model(roberta1, test_dataloader, test_y)
    # exp_results[f'No_Replace'] = res
    continue

  roberta1_st = roberta1.roberta.state_dict() # Bert model state dictionary
  y_tensor = roberta1_st[f"encoder.layer.{i-1}.intermediate.dense.weight"].detach().to(device) # get weights for ith ff layer

  # xes = run_omp(X_tensor.T,  y_tensor, n_non_zero) # run fast OMP
  reg = OrthogonalMatchingPursuit(normalize=False).fit(X_tensor.T,  y_tensor)
  xes = reg.predict(X_tensor.T)

  temp_weights = xes @ X_tensor # matrix mult to get the weights

  roberta1_st[f"encoder.layer.{i-1}.intermediate.dense.weight"]  = temp_weights
  roberta1.bert.load_state_dict(roberta1_st)
  print(f"Results for layer-{i-1} Weights Replacement")
  res = eval_model(roberta1)
  exp_results[f'layer-{i-1}'] = res


  y_tensor.to("cpu")

pd.DataFrame.from_dict(exp_results).to_excel(f'/content/drive/MyDrive/NLP-Final/experiments_results/Roberta/Replace_each_layer_leaving_others_untouched.xlsx')

# Replace Every layer one after the other

In [ ]:
# Add replace and test -  in every step 
emb1 = bert1.bert.get_input_embeddings() # embeddings
bert1_st = bert1.bert.state_dict() # Bert model state dictionary

X_tensor = emb1.weight.detach().to(device) 
num_features = X_tensor.shape[1]
n_non_zero = max(int(0.1 * num_features), 1) # 10% of X num of features

exp_results = dict()

for i in range(13):
  torch.cuda.empty_cache() # Clear memory
  bert1.load_state_dict(torch.load(STATE_PATH)) # load first the original weights

  if i == 0:
    print("Results Before Replacement")
    res = eval_model(bert1)
    exp_results[f'No_Replace'] = res
    continue

  # bert1_st is updated every step
  y_tensor = bert1_st[f"encoder.layer.{i-1}.intermediate.dense.weight"].detach().to(device) # get weights for ith ff layer

  xes = run_omp(X_tensor.T,  y_tensor, n_non_zero) # run fast OMP

  temp_weights = xes @ X_tensor # matrix mult to get the weights

  bert1_st[f"encoder.layer.{i-1}.intermediate.dense.weight"]  = temp_weights
  bert1.bert.load_state_dict(bert1_st)
  print(f"Results for Replacing all the layers up to layer-{i-1} (included)")
  res = eval_model(bert1)
  exp_results[f'layer0_to_{i}'] = res

pd.DataFrame.from_dict(exp_results).to_excel(f'/content/drive/MyDrive/NLP-Final/experiments_results/Roberta/Replace_layers_one_by_one.xlsx')

# Sparsity Levels Experiment - 1% - 45% - for each layer

In [ ]:
# 1%-45%
import gc 
emb1 = bert1.bert.get_input_embeddings() # embeddings
bert1_st = bert1.bert.state_dict() # Bert model state dictionary

X_tensor = emb1.weight.detach().to(device) 
num_features = X_tensor.shape[1]


ratios = [0.0,0.01,0.05,0.1,0.15,0.2,0.3,0.35,0.4,0.45]

for j in range (12):
  gc.collect()
  exp_results = dict()
  for i, ratio in enumerate(ratios):
    # Change ratio of non_zero_coef each iteration
    n_non_zero = max(int(ratio * num_features), 1) # ratio of X num of features
    torch.cuda.empty_cache() # Clear memory
    
    bert1.load_state_dict(torch.load(STATE_PATH)) # load first the original weights

    if i == 0: # print baseline results first
      print("Results Before Replacement")
      eval_model(bert1)
      continue

    bert1_st = bert1.bert.state_dict() # Bert model state dictionary
    y_tensor = bert1_st[f"encoder.layer.{j}.intermediate.dense.weight"].detach().to(device) # get weights for ith ff layer

    xes = run_omp(X_tensor.T,  y_tensor, n_non_zero) # run fast OMP

    temp_weights = xes @ X_tensor # matrix mult to get the weights

    bert1_st[f"encoder.layer.{j}.intermediate.dense.weight"]  = temp_weights
    bert1.bert.load_state_dict(bert1_st)
    print(f"Results for layer-{j} Weights Replacement with {100*ratio:.2f}% Non-Zero Coefficients")
    res = eval_model(bert1)
    exp_results[f'{100*ratio:.2f}%'] = res
    y_tensor.to("cpu")

  pd.DataFrame.from_dict(exp_results).to_excel(f'/content/drive/MyDrive/NLP-Final/experiments_results/Roberta/Sparsity_Levels_1_45_layer{j}.xlsx')
